In [23]:
!pip install datascience
!pip install wget
!pip install plotly
!pip install pdb

from datascience import *
from plotly.offline import plot
import wget
import plotly.plotly as py
import plotly.graph_objs as go
import math
import pdb
import numpy as np

  Could not find a version that satisfies the requirement pdb (from versions: )
No matching distribution found for pdb


In [13]:
wget.download('https://raw.githubusercontent.com/datasets/speed-dating/master/data/speed-dating.csv')

100% [............................................................................] 306068 / 306068

'sleeping-alone-data.csv'

# numpy.logical_and() and Table.where() #

In order to use `Table.where()` with multiple conditional arguments, you can use a combination of list comprehension and the function `numpy.logical_and()`, which does not raise a `ValueError` line `and` does when used in a list comprehension. `Table.where()` takes a list of booleans, so we can use a for loop to loop through both conditionals and output `True` if both are true, and vice versa. This lets us say:

`newTable = someTable.where([x for x in np.logical_and(someCondition1, someCondition2)])`

In [103]:
#Run to graph a user's rating of their intelligence against their speed dating partner's rating of the users intelligence, all over user ID

#Initiating the master table
dateData = Table.read_table('D:\Datasets\speed-dating.csv')

#Creates a table with just the two intelligence ranking columns, and a second table with a third column,
#the difference between the two rankings, sorted by that ranking
intelRanking = dateData.where([notnan for notnan in np.logical_and(np.isnan(dateData['intelligence_o']) == False, np.isnan(dateData['intelligence']) == False)]).select('intelligence_o', 'intelligence')
intByDiff = intelRanking.with_column('intDiff', np.abs(intelRanking['intelligence_o'] - intelRanking['intelligence'])).sort('intDiff', descending=True)

#Convenient x-axis label for graphing both ratings on the y
userID = list(range(intelRanking.num_rows))

#Formatting data for graphing
pList = [intXCoord for intXCoord in intByDiff['intelligence_o']]
sList = [intYCoord for intYCoord in intByDiff['intelligence']]

#Graphing data
intPPoints = go.Scatter(x=userID, y=pList, mode='markers', name='Intelligence as Ranked by Partner')
intSPoints = go.Scatter(x=userID, y=sList, mode='markers', name='Intelligence as Ranked by Self')
plot([intPPoints, intSPoints], filename='parIntOverSelfInt.html')

'file://C:\\Users\\Sivan Cooperman\\Desktop\\python\\Data-Science\\parIntOverSelfInt.html'

In [26]:
#Creating a dictionary with race key and frequency value
raceDict = {}
for race in dateData['race']:
    if race not in raceDict:
        raceDict[race] = 1
    else:
        raceDict[race] += 1

#Formatting data for graphing
raceLabels = list(raceDict)
numRace = [freq for race1, freq in raceDict.items()]
raceData = go.Pie(labels=raceLabels, values=numRace)

#Graphing data
plot([raceData], filename='raceDistDating.html')
        


'file://C:\\Users\\Sivan Cooperman\\Desktop\\python\\Data-Science\\raceDistDating.html'

In [96]:
#Tables of whites with non-white partners and whites with white partners
mixedWhiteDates = dateData.where('race', are.equal_to('european/caucasian-american')).where('samerace', are.equal_to(0))
whiteDates = dateData.where('race', are.equal_to('european/caucasian-american')).where('samerace', are.equal_to(1))

#mpa = mixedPercentAttractive, the percent of non-white partners rated a 7 or higher on attractiveness by whites
mpa = mixedWhiteDates.where('attractive_partner', are.above_or_equal_to(7)).num_rows/mixedWhiteDates.num_rows

#wpa = whitePercentAttractive, the percent of white partners rated a 7 or higher on attractiveness by whites
wpa = whiteDates.where('attractive_partner', are.above_or_equal_to(7)).num_rows/whiteDates.num_rows

#mrppa = mixedRacePreferentialPercentAttractive, the percent of non-white partners rated a 7 or higher on attractiveness
#by whites who rated importance of a same-race partner 7 or higher
mrppa = mixedWhiteDates.where('attractive_partner', are.above_or_equal_to(7)).where('importance_same_race',\
are.above_or_equal_to(7)).num_rows/mixedWhiteDates.where('importance_same_race', are.above_or_equal_to(7)).num_rows

#wrppa = whiteRacePreferentialPercentAttractive, the percent of white partners rated a 7 or higher on attractiveness
#by whites who rated importance of a same-race partner 7 or higher
wrppa = whiteDates.where('attractive_partner', are.above_or_equal_to(7)).where('importance_same_race',\
are.above_or_equal_to(7)).num_rows/whiteDates.where('importance_same_race', are.above_or_equal_to(7)).num_rows

#Formatting and labeling data for graphing
attractivenessByRaceLabels = ['Attractive non-white',
                              'Attractive white',
                              'Attractive non-white with same-race pref',
                              'Attractive white with same-race pref']
attractivenessByRacePercents = [mpa, wpa, mrppa, wrppa]
attractivenessByRacePercentsRounded = [round(x,3) for x in attractivenessByRacePercents]

attractivenessData = go.Bar(x=attractivenessByRaceLabels, y=attractivenessByRacePercentsRounded)

#Graphing data
plot([attractivenessData], filename='attractiveness.html')

'file://C:\\Users\\Sivan Cooperman\\Desktop\\python\\Data-Science\\attractiveness.html'